# Proyecto Big Data

In [ ]:
!pip install -r requirements.txt

In [67]:
import json
from pymongo import MongoClient
import os
import requests
import gzip
import shutil
import time

#### 1. Bajamos la data correspondiente

In [2]:
def download_data(link, carpeta_destino):
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    nombre_archivo = os.path.basename(link)
    ruta_comprimido = os.path.join(carpeta_destino, nombre_archivo)
    ruta_descomprimido = os.path.splitext(ruta_comprimido)[0] # Eliminar la extensión .gz

    response = requests.get(link, stream=True)
    if response.status_code == 200:
        with open(ruta_comprimido, 'wb') as f:
            f.write(response.content)
            print(f'Archivo descargado: {ruta_comprimido}')
    else:
        print(f'Error al descargar el archivo: {ruta_comprimido}')
        return None

    with gzip.open(ruta_comprimido, 'rb') as archivo_comprimido:
        with open(ruta_descomprimido, 'wb') as archivo_descomprimido:
            shutil.copyfileobj(archivo_comprimido, archivo_descomprimido)
    print(f'Archivo descomprimido: {ruta_descomprimido}')

    return ruta_descomprimido

link_gift_cards = "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Gift_Cards.jsonl.gz"
link_meta_gift_cards = "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/meta_Gift_Cards.jsonl.gz"
carpeta_destino = "data/raw"
ruta_final_gift_cards = download_data(link_gift_cards, carpeta_destino)
ruta_final_meta_gift_cards = download_data(link_meta_gift_cards, carpeta_destino)

Archivo descargado: data/raw\Gift_Cards.jsonl.gz
Archivo descomprimido: data/raw\Gift_Cards.jsonl
Archivo descargado: data/raw\meta_Gift_Cards.jsonl.gz
Archivo descomprimido: data/raw\meta_Gift_Cards.jsonl


#### 2. Incorporamos la data en Mongo DB

Importante: Crear una conexion en MongoDB con los siguientes nombres de coleccion (gift-cards & meta-gift-cards)

In [3]:
# Conectar a MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Amazon"]
collection = db["gift-cards"]
collection_meta = db["meta-gift-cards"]

2.1 Gift Cards

In [4]:
# Borramos toda la data antes de volver a insertarla
collection.delete_many({})
print("Todos los documentos de la colección han sido eliminados.")

# Leer el archivo JSON Lines
with open("data/raw/Gift_Cards.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        try:
            data = json.loads(line)
            # Insertamos la data
            collection.insert_one(data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar la línea: {e}")

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 674aa5f45232a9d255180b95, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

2.2 Meta Gift Cards

In [7]:
# Borramos toda la data antes de volver a insertarla
collection_meta.delete_many({})
print("Todos los documentos de la colección han sido eliminados.")

# Leer el archivo JSON Lines
with open("data/raw/meta_Gift_Cards.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        try:
            data = json.loads(line)
            # Insertamos la data
            collection_meta.insert_one(data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar la línea: {e}")

Todos los documentos de la colección han sido eliminados.


#### 3. Querys MongoDB

3.1 Consulta para actualizar el campo "main_category" a "Gift Card" de la coleccion de Meta Gift Card

In [3]:
result = collection_meta.update_many({}, {"$set": {"main_category": "Gift Cards"}})
print("Documentos actualizados")

Documentos actualizados


3.2 Consulta en donde se busquen los comentarios que tienen un verified_purchase con valor True, ademas que tengan un rating mayor o igual a 1 y menor o igual a 5, y un helpful_vote mayor o igual a 0

In [10]:
query = {
    "verified_purchase": True,
    "rating": {"$gte": 1, "$lte": 5},
    "helpful_vote": {"$gte": 0},
}

results = collection.find(query)

#### 4. Dask y Pandas

4.1 Limpieza de data

In [ ]:
import re
from html import unescape
import codecs

def clean_text(text):
    if not isinstance(text, str):
        return text
    text = unescape(text) # html entities?

    # text = codecs.decode(text, 'unicode_escape') #decode emojis
    text = re.sub(r'\\u[0-9a-fA-F]{4}', '', text) #remove emojis

    text = re.sub(r'!{2,}', '!', text) #Multiple exclamation

    text = re.sub(r'\.{2,}', ' ', text) # Dots

    text = re.sub(r'<br\s*/?>', ' ', text)  # Para br

    text = re.sub(r'\b([A-Z])(?:\s+([A-Z]))+\b', lambda m: m.group(0).replace(" ", "").lower(), text) # t h i s to this

    text = re.sub(r'[+-]', '', text) #omit some chars

    text = re.sub(r'\u2019', "'", text) # this ´ to this '

    text = re.sub(r'(^|\.\s+)([a-z])', lambda m: m.group(1) + m.group(2).upper(), text) #Capital 
    
    text = re.sub(r'\b[A-Z]{2,}\b', lambda m: m.group().lower(), text) #Words normalized
    
    return text

4.2 Medicion de tiempos

In [68]:
import pandas as pd

file_path = "data/raw/Gift_Cards.jsonl"

start_time = time.time()

df = pd.read_json(file_path, lines=True)

df["title"] = df["title"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)

# pd.set_option("max_colwidth", None)
# print(df.loc[189])

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

Time taken: 4.28 seconds


In [69]:
import dask.dataframe as dd

file_path = "data/raw/Gift_Cards.jsonl"

start_time = time.time()

df = dd.read_json(file_path, lines=True)

df["title"] = df["title"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)

end_time = time.time()

# print(df.loc[189].compute())

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

Time taken: 1.26 seconds


c:\Users\Jose\Desktop\amazon-recommender-system\venv\Lib\site-packages\dask_expr\_collection.py:4380: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'object'))

  warnings.warn(meta_warning(meta))
c:\Users\Jose\Desktop\amazon-recommender-system\venv\Lib\site-packages\dask_expr\_collection.py:4380: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  Af

#### 5. Seteamos LLM